In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import faiss
from langchain.vectorstores import FAISS
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["HUGGINGFACE_TOKEN"]=os.getenv("HUGGINGFACE_TOKEN")

In [3]:
pdf_loader=PyPDFLoader("llama2.pdf")
pdf_doc=pdf_loader.load()
len(pdf_doc)

77

In [4]:
pdf_spliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [5]:
splited_text=pdf_spliter.split_documents(pdf_doc)
len(splited_text)

615

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Hp\miniconda3\envs\agentic_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.docstore.in_memory import InMemoryDocstore
index=faiss.IndexFlatIP(384)
vector_store=FAISS(embedding_function=embeddings,index=index,
                   docstore=InMemoryDocstore(),
                   index_to_docstore_id={})

In [9]:
vector_store.add_documents(splited_text)

['192c6d2f-638b-4ec1-8a56-0525ad2392fa',
 '9056697c-a202-47cd-80ee-f99ba25134fe',
 '80bb8a14-e21b-46db-a237-0dc844743e0e',
 'c626438f-573e-49d8-8cd5-077dbbdae542',
 'db630850-d665-478d-8fdb-565201027007',
 '1f9cb927-36f2-469e-a4d7-cff91bdc7dc0',
 '38ca05c1-aa0a-47c5-bf89-afb08e9c11a2',
 'bfc23dd2-e1c3-4b26-a528-71e42070a11f',
 'af4e711f-4c29-4772-a461-3b09e2054a25',
 '3fa25ecd-b61b-4c26-9212-0fc27ddcb18c',
 'da6a9fd5-b9ef-4c17-9235-8177a5da27a5',
 '379e1451-57d8-40a1-8ba6-7b250a729304',
 '73ce157b-22aa-463e-b24c-b9f8533d2e3a',
 'bc799534-524d-4d43-98ed-fcd67708d2d4',
 '42ea6482-53d0-446b-9563-d4f4b0c619b3',
 'ccad1c6a-9974-4221-9946-a895f1f59671',
 '96d2907f-38c2-4130-ab3f-da7df0d9cb47',
 '39b26490-a66c-42aa-aa32-3ce6a6db0d9b',
 '7503eab3-2585-4310-88f0-d22333ca5ea3',
 'c17acc72-c183-4641-ac0d-c5b821185db3',
 '4fd60620-6890-43f6-8f31-12455890d390',
 '98e28848-307f-40ce-b94b-a6588c29ef89',
 '9e10be29-4cc1-47c5-8e5e-b1e2a330c2b1',
 'f0507e27-d1ca-4da7-a56a-5844fc6f056b',
 '7712d132-6be8-

In [11]:
#Vector store working as a similarity search engine
similar_data=vector_store.similarity_search("What is lamma model?",k=2)
similar_data

[Document(id='bfdcdd44-0ba2-437b-ae1d-663e0d9cdb0e', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'llama2.pdf', 'total_pages': 77, 'page': 76, 'page_label': '77'}, page_content='specific applications of the model. Please see the Responsible Use Guide available available at\nhttps://ai.meta.com/llama/responsible-user-guide\nTable 52: Model card forLlama 2.\n77'),
 Document(id='23d73782-9ec1-4a74-9f28-98de4eeb2e25', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 202

In [38]:
#Working as a retriever

retriver=vector_store.as_retriever(search_kwargs={"k":30}) 

In [39]:
result=retriver.invoke("What is langchain framework?")

In [40]:
len(result)

30

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI

model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [30]:
from langchain import hub
prompt=hub.pull('rlm/rag-prompt')

In [31]:
import pprint
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough   # used for take query only in run time

In [33]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [34]:
chain=(
    {"context": retriver | format_docs,"question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser())

In [35]:
result=chain.invoke("What is llama model?")

In [36]:
result

'Llama is a family of large language models (LLMs) developed by Meta AI.  It comes in various sizes (7B, 13B, 70B parameters) and includes both pretrained and fine-tuned versions.  The fine-tuned versions, like Llama 2-Chat, are optimized for assistant-like chat applications.'

In [41]:
result=chain.invoke("What is langchain?")
result

'I am sorry, but this document does not contain any information about Langchain.  Therefore, I cannot answer your question.'